In [1]:
import uproot
import re
import awkward as ak
import numpy as np

from cfg.hnl_mva_tools import read_json_file
from data_tools.load_data import filter_trees

In [3]:

json_file = read_json_file("cfg/ntuples.json")
signal_files = json_file["signal"]
new_signal_files = json_file["signal"]
background_files = json_file["background"]
treename = json_file["treename"]
weight_name = json_file["weight_name"]

#list variables
good_vars = read_json_file("cfg/vars_old.json")["vars"]
#add weight and event (normally not included in variables because they are not variables to train on)
good_vars.append(weight_name)
good_vars.append("event")


#you can choose my_signal_files and my_new_signal_files if you want to
#modify only some specific files
my_signal_files = ["/home/quibus/hnl_ntuples_for_mva/tree_HnlToMuPi_prompt_DsToNMu_NToMuPi_SoftQCDnonD_noQuarkFilter_mN1p0_ctau10p0mm_TuneCP5_13TeV-pythia8-evtgen_tree.root"]
my_new_signal_files = ["mN1p0"]
for signal_file,new_signal_file in zip(my_signal_files,my_new_signal_files):
    #open the root file and tree with uproot
    file = uproot.open(signal_file)
    tree = file[treename]

    #load the variables you want to use
    events = tree.arrays(good_vars, library="ak", how=dict)
    my_arr = events["C_pass_gen_matching"]
    number = 0
    for element in my_arr:
        #element is an array
        #if it only contains zeros it means that the event did not pass the gen matching
        #in that case increase number by 1
        if np.all(element==0):
            number += 1
    print(f"number of events that did not pass gen matching: {number}")
    print(f"total number of events: {len(my_arr)}")
    break


number of events that did not pass gen matching: 51
total number of events: 9834
